In [1]:
# 📊 STEP 1: Load trade journal
import sys
sys.path.append("../") 

import json
from pathlib import Path

journal_path = Path("data/journal.json")

if journal_path.exists():
    with open(journal_path, "r") as f:
        journal = json.load(f)
    print(f"✅ Loaded {len(journal)} journal entries.")
else:
    print("❌ No journal found. Run previous steps to generate trade logs.")
    journal = []


✅ Loaded 44 journal entries.


In [2]:
# 📘 STEP 2: Prepare trade history for GPT (with safeguards)
explanations = [entry["explanation"] for entry in journal if "explanation" in entry]
reflections = [
    entry["reflections"]
    for entry in journal
    if "reflections" in entry
    and isinstance(entry["reflections"], dict)
    and all(k in entry["reflections"] for k in ("mindset", "rules_followed", "lesson"))
]

trade_history_summary = "\n\n".join(explanations[:10])
mindset_summary = "\n".join([
    f"- 🧠 {r['mindset']}, 📏 Rules: {r['rules_followed']}, 📚 Lesson: {r['lesson']}"
    for r in reflections
])

print("📖 Preview GPT Input (trades + mindset):\n")
print(trade_history_summary[:1000])  # preview



📖 Preview GPT Input (trades + mindset):

Sure, let's break down the calendar call spread strategy in simple terms:

**What is a Calendar Call Spread?**
A calendar call spread involves buying and selling call options with the same strike price but different expiration dates.

**Trade Details:**
- Ticker: SPY is the ETF representing the S&P 500 index.
- Strategy: Calendar call spread means buying a longer-term call option and selling a shorter-term call option.
- Strike Price: Both the short and long call options have a strike price of 520.0.
- Expiration Dates: The short call expires on April 7, 2025, and the long call expires on April 8, 2025.
- Net Debit: The trade costs 1.62 per share to enter.

**Profit Potential:**
- The goal of a calendar call spread is to profit from the difference in time decay between the short and long options.
- If the stock price stays near the strike price (520.0) until the short option expires, the trade can potentially benefit from the accelerated time de

In [3]:
# STEP 3: Build a strategy using GPT

from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the client
client = OpenAI(api_key=api_key)

# Compose your prompt
prompt = f"""
You are an elite options strategist and performance coach.

Analyze the following trading history and mindset reflections to identify patterns of success.
Then, suggest **3 new option strategies** that align with the trader's strengths.

⚠️ Use this exact format below — no extra text, emojis, or markdown formatting:

1. 🗂 Strategy Name: <strategy name>
   - Description: <short explanation>
   - Why this fits the trader: <reason>
   - Best conditions for this strategy: <market conditions>

2. 🗂 Strategy Name: <strategy name>
   - Description: <short explanation>
   - Why this fits the trader: <reason>
   - Best conditions for this strategy: <market conditions>

3. 🗂 Strategy Name: <strategy name>
   - Description: <short explanation>
   - Why this fits the trader: <reason>
   - Best conditions for this strategy: <market conditions>

--- TRADE EXPLANATIONS ---
{trade_history_summary}

--- MINDSET REFLECTIONS ---
{mindset_summary}
"""



# Call GPT with the new syntax
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You're a trading mentor and strategy optimizer."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

# Extract the result
strategy_output = response.choices[0].message.content.strip()



In [4]:
# 📤 STEP 4: Output the GPT-generated strategies
print("📌 GPT Strategy Recommendations:\n")
print(strategy_output)

# Convert raw string output into a structured list of strategies
# For now, we're using a simple parser – you can replace this with regex or GPT parsing later
strategies_list = []

blocks = strategy_output.split("1. ")[-1].split("\n2. ")
blocks = [f"1. {blocks[0]}"] + [f"2. {blocks[1]}"] if len(blocks) == 2 else [f"1. {blocks[0]}"]
if "3. " in blocks[-1]:
    last = blocks.pop()
    second, third = last.split("\n3. ")
    blocks.append(second)
    blocks.append(third)

for block in blocks:
    lines = block.strip().split("\n")
    strategy = {
        "strategy_name": lines[0].split(":", 1)[-1].strip(),
        "description": "",
        "why_it_fits": "",
        "best_conditions": ""
    }
    for line in lines[1:]:
        if "- Description:" in line:
            strategy["description"] = line.split(":", 1)[-1].strip()
        elif "- Why this fits the trader:" in line:
            strategy["why_it_fits"] = line.split(":", 1)[-1].strip()
        elif "- Best conditions for this strategy:" in line:
            strategy["best_conditions"] = line.split(":", 1)[-1].strip()
    strategies_list.append(strategy)

# ✅ Save structured list to JSON
from pathlib import Path
import json

output_path = Path("data/strategies_generated.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    json.dump({"strategies": strategies_list}, f, indent=2)

print(f"\n✅ Strategies saved to {output_path}")



📌 GPT Strategy Recommendations:

1. 🗂 Strategy Name: Iron Condor
   - Description: An Iron Condor involves selling a call spread and a put spread on the same underlying asset with the same expiration date.
   - Why this fits the trader: This trader is experienced with spreads and has shown an ability to manage trades with multiple legs. The Iron Condor offers a defined risk-reward setup, which aligns with this trader's risk management skills. 
   - Best conditions for this strategy: Iron Condors work best in a range-bound market where the underlying asset is expected to move within a specific price range.

2. 🗂 Strategy Name: Butterfly Spread
   - Description: A Butterfly Spread involves buying a long call at a lower strike price, selling two calls at a middle strike price, and buying another long call at a higher strike price. All options have the same expiration date.
   - Why this fits the trader: The trader has shown an understanding of the time decay concept (Theta) in options tra